In [1]:
from icecream import ic

In [2]:
sen1 = "?P needs ?X"
sen2 = "John needs vacation"

In [3]:
def is_variable(s):
    return s.startswith("?") and all(i.isalpha() for i in s[1:])

In [4]:
def pat_match(pattern, saying):
    if not pattern or not saying:
        return []
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]:
            return []
        return pat_match(pattern[1:], saying[1:])

In [5]:
got_patterns = pat_match("I want ?X and ?XX".split(), "I want holiday and chocolate".split())

In [6]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

In [7]:
def substitute(rule, parsed_rules):
#     ic(rule, parsed_rules)
    if not rule:
        return []
    return [parsed_rules.get(rule[0], rule[0])] + substitute(rule[1:], parsed_rules)

In [8]:
john_pat = pat_match(sen1.split(), sen2.split())

In [9]:
john_pat

[('?P', 'John'), ('?X', 'vacation')]

In [10]:
' '.join(substitute("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

'Why does John need vacation ?'

In [11]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [12]:
import random

In [13]:
def get_response(saying):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    for pat in defined_patterns.keys():
        pm = pat_match(pat.split(), saying.split())
#         ic(pm)
        if pm:
            dp = random.choice(defined_patterns.get(pat))
            return ' '.join(substitute(dp.split(), pat_to_dict(pm)))

In [14]:
get_response("I need iPhone")

'Why do you need iPhone ?'

In [15]:
get_response("My mother told me something")

'Talk about more about your mother'

In [16]:
def is_pattern_segment(pattern: str):
    return pattern.startswith("?*") and all(a.isalpha() for a in pattern[2:])

In [17]:
is_pattern_segment('?*sadfsad')

True

In [18]:
from collections import defaultdict

In [19]:
fail = None

def pat_match_with_seg(pattern, saying):
#     ic(pattern, saying)
    if not pattern or not saying:
        return []
    pat = pattern[0]
    if is_variable(pat):
#         ic(pattern[1:], saying[1:])
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
#         ic(match, index)
        if index != -1:
            return [match] + pat_match_with_seg(pattern[1:], saying[index:])
        # 没有匹配到
        return {}
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [20]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace("?*", '?')
    
    if not rest:
        return (seg_pat, saying), len(saying)
    
    for i, token in enumerate(saying):
#         ic(rest[0], token)
        if rest[0] == token and is_match(rest[1:], saying[i+1:]):
            return (seg_pat, saying[:i]), i
        
    return ([], []), -1

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [21]:
is_match("abc 1X".split(), "abc bg".split())

True

In [22]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [23]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [24]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [25]:
substitute("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', ['an', 'iPhone']]

In [26]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [27]:
substitute("Were you really ?X ?".split(), pat_to_dict(pat_match_with_seg('I was ?*X'.split(), 
                  "I was from China".split())))

['Were', 'you', 'really', 'from China', '?']

In [28]:
pat_match_with_seg("?*x hello ?*y".split(), "Today I feel happier than yesterday".split())

{}

In [29]:
substitute("Please state your problem".split(), pat_to_dict(pat_match_with_seg("?*x hello ?*y".split(),
                "Today I feel happier than yesterday".split())))

['Please', 'state', 'your', 'problem']

In [30]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

### 问题1

In [31]:
def get_response(saying, response_rules):
    for pat in response_rules.keys():
        pm = pat_match_with_seg(pat.split(), saying.split())
#         ic(pm)
        if pm:
            dp = random.choice(response_rules.get(pat))
            return ' '.join(substitute(dp.split(), pat_to_dict(pm)))

In [32]:
get_response("Today I feel happier than yesterday", rule_responses)

'Do you often feel happier than yesterday ?'

### 问题2

In [33]:
def is_Chinese(word):
    for ch in word:
        if '\u4e00' <= ch <= '\u9fff':
            return True
    return False

import jieba

def my_split(word):
    if is_Chinese(word):
        tmp = list(jieba.cut(word, cut_all=False))
        ret = []
        while len(tmp):
            tmp_s = "".join(tmp[:3])
            if tmp_s in ["?*x" , "?*y", "?*z"]:
                ret += [tmp_s]
                tmp = tmp[3:]
            else:
                tmp_s = "".join(tmp[:2])
                if tmp_s in ["?x", "?y", "?z"]:
                    ret += [tmp_s]
                    tmp = tmp[2:]
                else:
                    ret += [tmp[0]]
                    tmp = tmp[1:]
        return ret
    return word.split()

def get_response(saying, response_rules):
    for pat in response_rules.keys():
#         ic(my_split(pat), my_split(saying))
        pm = pat_match_with_seg(my_split(pat), my_split(saying))
#         ic(pm)
        if pm:
#             ic(pm)
            dp = random.choice(response_rules.get(pat))
            if is_Chinese(pat):
                return ''.join(substitute(my_split(dp), pat_to_dict(pm))).replace(" ", '')
            else:
                return ' '.join(substitute(my_split(dp), pat_to_dict(pm)))

In [34]:
# my_split("?*x你好?*y")
get_response("有的人走了，他还活着", rule_responses)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.534 seconds.
Prefix dict has been built succesfully.


'具体是哪些人呢?'

In [35]:
get_response("你确定你不要吃冰淇淋吗？", rule_responses)

'你有一点负能量'

In [36]:
get_response("你说你为什么不吃饭", rule_responses)

'等我心情好了，我就吃饭'

In [38]:
get_response("我和你说，我昨天梦见了她", rule_responses)

'你以前梦见过了她吗'

### 问题4

1. 优点是匹配速度快；缺点是不够完善，只能匹配库中有的信息，要去丰富库内容。最好可以依据上下文动态完善库，就和真实对话场景更接近。
2. 数据驱动应该指的是通过海量的数据形成算法模型，再把数据的结果反馈回去。这个程序里面，通过我们输入的数据给出相应的回答，体现了数据驱动。
3. 数据驱动让AI有学习能力，从而更智能。